In [3]:
import cv2
import numpy as np
import tensorflow as tf
import pickle

In [4]:
MODEL_PATH = 'models/garbage_classifier_v4_balanced.h5'
CLASS_NAMES_PATH = 'models/class_names.pkl'
IMAGE_SIZE = (224, 224)
CONFIDENCE_THRESHOLD = 75.0 

try:
    model = tf.keras.models.load_model(MODEL_PATH)
    with open(CLASS_NAMES_PATH, 'rb') as f:
        class_names = pickle.load(f)
except Exception as e:
    print(f"Tidak bisa memuat model {e}")
    exit()

recyclability_map = {
    'battery': ('Non-Recyclable', (0, 0, 255)),
    'biological': ('Non-Recyclable', (0, 0, 255)),
    'brown-glass': ('Recyclable', (0, 255, 0)),
    'cardboard': ('Recyclable', (0, 255, 0)),
    'clothes': ('Recyclable', (0, 255, 0)),
    'green-glass': ('Recyclable', (0, 255, 0)),
    'metal': ('Recyclable', (0, 255, 0)),
    'paper': ('Recyclable', (0, 255, 0)),
    'plastic': ('Recyclable', (0, 255, 0)),
    'shoes': ('Non-Recyclable', (0, 0, 255)),
    'trash': ('Non-Recyclable', (0, 0, 255)),
    'white-glass': ('Recyclable', (0, 255, 0))
}

cap = cv2.VideoCapture(1)
if not cap.isOpened():
    print("Kamera Error")
    exit()

while True:
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)

    h, w, _ = frame.shape
    box_size = 350
    x1 = (w - box_size) // 2
    y1 = (h - box_size) // 2
    x2 = x1 + box_size
    y2 = y1 + box_size

    cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 255), 2)
    
    roi = frame[y1+2:y2-2, x1+2:x2-2]

    img_resized = cv2.resize(roi, IMAGE_SIZE)
    img_array = np.expand_dims(img_resized, axis=0)

    prediction = model.predict(img_array, verbose=0)
    confidence = np.max(prediction[0]) * 100

    if confidence > CONFIDENCE_THRESHOLD:
        predicted_class_index = np.argmax(prediction[0])
        predicted_class_name = class_names[predicted_class_index]
        recyclable_status, status_color = recyclability_map.get(predicted_class_name, ("N/A", (255, 255, 255)))
        
        cv2.rectangle(frame, (x1, y1), (x2, y2), status_color, 3)
        
        text_class = f"Jenis: {predicted_class_name}"
        text_confidence = f"Keyakinan: {confidence:.1f}%"
        text_status = f"Status: {recyclable_status}"
    else:
        text_class = "Arahkan sampah ke dalam kotak!"
        text_confidence = ""
        text_status = ""

    (text_width, text_height), _ = cv2.getTextSize(text_class, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 1)
    max_text_width = box_size - 20 
    
    font_scale = 0.6
    if text_width > max_text_width:
        font_scale = max_text_width / text_width * 0.6
    
    text_bg_height = 80 if text_status else 50
    overlay = frame.copy()
    cv2.rectangle(overlay, (x1, y1-5), (x2, y1+text_bg_height), (0, 0, 0), -1) 
    frame = cv2.addWeighted(overlay, 0.5, frame, 0.5, 0)

    font_thickness = 1
    text_color = (255, 255, 255)
    line_height = int(25 * font_scale/0.6)
    
    cv2.putText(frame, text_class, (x1 + 10, y1 + 25), 
               cv2.FONT_HERSHEY_SIMPLEX, font_scale, text_color, font_thickness)
    
    if text_confidence:
        cv2.putText(frame, text_confidence, (x1 + 10, y1 + 25 + line_height), 
                   cv2.FONT_HERSHEY_SIMPLEX, font_scale, text_color, font_thickness)
    
    if text_status:
        cv2.putText(frame, text_status, (x1 + 10, y1 + 25 + 2*line_height), 
                   cv2.FONT_HERSHEY_SIMPLEX, font_scale, text_color, font_thickness)

    cv2.imshow('Garbage Classifier - Tekan "q" untuk keluar', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()